In [157]:
import callbacks
import numpy as np

In [138]:
MASTER_ALL, MASTER_PID, DATE_MAPPER, KEY_VALUE = callbacks.serve_data(ret=True)

Grabbing from S3


In [159]:
MASTER_ALL['per_capita_confirmed'] = MASTER_ALL['confirmed']/MASTER_ALL['pop']
MASTER_ALL['per_capita_deaths'] = MASTMER_ALL['deaths']/MASTER_ALL['pop']

max_size=1500

bins, ret_bins = pd.qcut(MASTER_ALL[(MASTER_ALL['per_capita_confirmed'] >= 0) & (MASTER_ALL['country'] != 'worldwide')]['per_capita_confirmed'], q=[
    0, .5, 0.6, 0.70, 0.75, 0.8, 0.85, 0.9, 0.95, 0.999, 1], duplicates='drop', retbins=True)
yellows = ["#606056", "#6e6e56", "#7b7c55", "#898a54", "#979953",
           "#a5a850", "#b4b74d", "#c3c649", "#d2d643", "#e2e53c"]
reds = ["#5a4f4f", "#704d4d", "#854b4a", "#994746", "#ac4340",
        "#be3c3a", "#cf3531", "#e02a27", "#f01c19", "#ff0000"]
labels = np.geomspace(1, max_size, num=len(ret_bins)-1)
MASTER_ALL['per_capita_CSize'] = pd.cut(MASTER_ALL['per_capita_confirmed'], bins=ret_bins, labels=labels).astype(float).fillna(0)
MASTER_ALL['per_capita_DSize'] = pd.cut(MASTER_ALL['per_capita_deaths'], bins=ret_bins, labels=labels).astype(float).fillna(0)
MASTER_ALL['per_capita_CColor'] = pd.cut(MASTER_ALL['per_capita_confirmed'], bins=ret_bins,
                              labels=yellows[2:]).astype(str).replace({'nan': 'white'})
MASTER_ALL['per_capita_DColor'] = pd.cut(MASTER_ALL['per_capita_deaths'], bins=ret_bins,
                              labels=reds[2:]).astype(str).replace({'nan': 'white'})

MASTER_ALL['Text_Confirmed'].str.split(':').str.get(
                    0) + ":1 in " + (1/MASTER_ALL['per_capita_confirmed']).replace(
                        np.inf, 0).astype(int).apply(lambda x: "{:,}".format(x))